# Elliptic Curve Digital Signature Algorithm
This is an implementation of the algorithm that allows crypto currency to work. 

In [1]:
import hashlib
import random

In [2]:
def invMod(a,n):
    #this finds the inverse of a mod n
    r1 = n
    r2 = a%n
    tempR = 0
    t1 = 0  
    t2 = 1  
    tempT = 0
    q = 0
    while (r2 > 0):        
        q = r1//r2
        tempT = t2
        t2 = t1 - q*t2
        t1 = tempT
        tempR = r2
        r2 = r1 - q*r2
        r1 = tempR
    if (r1 > 1):
        return 0
    return t1

In [3]:
def isPtOnC(P, Elist, char):
	x = P[0]
	y = P[1]
	a = Elist[0]
	b = Elist[1]
	c = Elist[2]
	if ( x == float("inf") and y == float("inf")):
		return True
	if ( (y**2 - x**3 - a*x**2 - b*x - c)%char == 0):
		return True
	else:
		return False

In [4]:
def doublePt(P, Elist, char):
	x = P[0]
	y = P[1]
	a = Elist[0]
	b = Elist[1]
	c = Elist[2]
	inf = float("inf")
	if (y == inf):
		return [inf,inf]
	if (y%char == 0): #first handle the case if our tangle line is vertical
		return [inf,inf]
	#now do the real work, remember
	#A = (3x^2 + 2ax + b)/(2y), so we need to invert 2y
	if (char > 0):  #first do the positive characteristic case
		twoyinv = invMod(2*y, char)
		if (twoyinv == 0):  #this shouldn't happen as long as our prime p > 2
			print "Error.  Couldn't invert the denominator."
			return False
		A = ((3*x**2 + 2*a*x+b)*twoyinv)%char
		
		#so we plug in y = A(x - x_0) + y_0 into our y^2 = x^3 + ... equation, and we observe that this becomes
		#x^3 + (a - A^2)x^2 + ...
		#then the x-coordinate of the intersection point is -a + A^2 - 2*x
		x2P = (A**2-a-2*x)%char
		y2P = (-(A*(x2P-x)+y))%char
		return [x2P, y2P]
	else: #now we do the characteristic zero case, one should really work with x and y in the fractions class (floats are not so great here).
		A = (3*x**2 + 2*a*x+b)/(2*y)
		x2P = (A**2 - a- 2*x)
		y2P = (-(A*(x2P-x)+y))
		return [x2P, y2P]

SyntaxError: Missing parentheses in call to 'print'. Did you mean print("Error.  Couldn't invert the denominator.")? (<ipython-input-4-bb77872fe9a9>, line 17)

In [5]:
def addPts(P, Q, Elist, char):	
	xP = P[0]
	yP = P[1]
	xQ = Q[0]
	yQ = Q[1]
	a = Elist[0]  #y^2 = x^3 + ax^2 + bx + c
	b = Elist[1]
	c = Elist[2]
	inf = float("inf")
	#first we handle all the easy cases
	if (yP == inf): 
		return Q
	if (yQ == inf):
		return P
	if (xP == xQ):
		if (yP == yQ):#if we are doubling a point...
			return doublePt(P, Elist, char)
		else: #if we are not doubling a point, but the x-coords
		      #agree, then we are going to add to infinity.
			return [inf, inf]
	#next do the real work.  
	if (char > 0):  #do the positive characteristic case
		
		inv=invMod(xP-xQ,char)
		A = ((yP-yQ)*inv)%char
		xPQ = (A**2 - a - xP - xQ)%char
		yPQ = (-(A*(xPQ-xP)+yP))%char
		return [xPQ, yPQ]
	else:
		A = (yP-yQ)/(xP-xQ)
		xPQ = (A**2 - a - xP - xQ)
		yPQ = (-(A*(x2P-x)+y))
		return [xPQ, yPQ]

In [6]:
def multPt(n, P, Elist, char):
	curN = n
	curP = P
	inf = float("inf")
	totalP = [inf,inf]
	
	if (curN == 0):
		return [inf, inf]
	if (curN == 1):
		return P
	if (curN < 0):
		return multPts(-curN, [P[0], -P[0]], Elist, char)
	while (curN > 0):
		#print curN
		#print curP
		if (curN%2 == 1):
			totalP = addPts(totalP, curP, Elist, char)
			#print "added"
			#print totalP
		curP = doublePt(curP, Elist, char)				
		curN = curN/2
		
	return totalP

In [ ]:
def HashMessage(message,n):
    h = hashlib.sha256()
    h.update(message)
    s = h.hexdigest()
    val = (int(s,16))%(n-2)
    return val

In [7]:
def sign(m,d,Q,n,Elist,char):
    #m is the message
    #d is the private key
    #Q is my point
    #n = ord(Q) is the prime number computed elsewhere
    #Elist is elliptic curve
    #char is the characteristic
    z = HashMessage(m,n)
    myRand = random.SystemRandom() #cryptographically secure random number
    r = 0
    while(r == 0):
        k = myRand.randint(1,n-1)
        kQ = multPt(k,Q,Elist,char) #computes k*Q
        x1 = kQ[0]
        r = x1%n
        kinv = invMod(k,n)
        s = (kinv*(z+ (r*d)))%n
    return [r,s]

In [8]:
def validate(m,R,Q,n,Elist, char, signature):
    #m,Q,n,Elist,char are same from above
    #R is d*Q Alice's public key
    #signature is the list [r,s]
    r = signature[0]
    s = signature[1]
    z = HashMessage(m,n)
    w = invMod(s,n)
    u1 = (z*w)%n
    u2 = (r*w)%n
    k1 = multPt(u1,Q,Elist,char)
    k2 = multPt(u2,R,Elist,char)
    T = addPts(k1,k2,Elist,char)
    return ((T[0])%n == r%n)